In [1]:
import numpy as np
import pandas as pd
import sklearn
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from IPython.display import display
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score


In [2]:
data = pd.read_csv("nba.games.stats.csv")
#pd.plotting.scatter_matrix(data[['TeamPoints','OpponentPoints','FieldGoals','X3PointShots.','Opp.Assists','Opp.Turnovers',"Home"]],figsize = (10,10))
data["Arena"] = pd.factorize(data.Home)[0]
data = data.drop(['Unnamed: 0','Game','Date','Home'],1)
data = data.loc[(data['Team'] == "TOR")]
data["Opponent"] = pd.factorize(data.Opponent)[0]


In [3]:
X_all = data.drop(['WINorLOSS','Team','Opponent'],1)
Y_all = data['WINorLOSS']
cols = [['TeamPoints','OpponentPoints','FieldGoals','X3PointShots.','Opp.Assists','Opp.Turnovers']]
for col in cols:
    X_all[col] = scale(X_all[col])
    


In [4]:
x_train,x_test,y_train,y_test = train_test_split(X_all,Y_all,
                                                 test_size = 100,
                                                 random_state = 42,
                                                 stratify = Y_all
                                                 
                                                )

In [19]:
#Classifiers:
"""linear = LogisticRegression(random_state = 42)
support_vector = SVC(random_state = 42, kernel = 'rbf' )
xgb_class = xgb.XGBClassifier(seed = 82)"""
def train_classifier(clf, x_train, y_train):
    clf.fit(x_train,y_train)

def predict_labels(clf, features, target):
    y_predict = clf.predict(features)
    return f1_score(target,y_predict,pos_label = 'W'),sum(target == y_predict)/float(len(y_predict))
#f1_score(target, y_pred, pos_label='H'), sum(target == y_pred) / float(len(y_pred))
                    
def train_predict(clf, x_train, y_train, x_test, y_test):
    train_classifier(clf, x_train, y_train)
    f1, acc = predict_labels(clf, x_train, y_train)
    print (f1, acc)
    print ("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, x_test, y_test)
    print ("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))
                    

In [20]:
'''linear.fit(x_train,y_train)
acc = linear.score(x_test,y_test)

support_vector.fit(x_train,y_train)
acc_2 = support_vector.score(x_test,y_test)

xgb_class.fit(x_train,y_train)
acc_3 = xgb_class.score(x_test,y_test)'''

# Initialize the three models (XGBoost is initialized later)
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 912, kernel='rbf')
#Boosting refers to this general problem of producing a very accurate prediction rule 
#by combining rough and moderately inaccurate rules-of-thumb
clf_C = xgb.XGBClassifier(seed = 82)

train_predict(clf_A, x_train, y_train, x_test, y_test)
print ('')
train_predict(clf_B, x_train, y_train, x_test, y_test)
print ('')
train_predict(clf_C, x_train, y_train, x_test, y_test)
print ('')


C:\Users\14372\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\14372\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.9932885906040269 0.9912280701754386
F1 score and accuracy score for training set: 0.9933 , 0.9912.
F1 score and accuracy score for test set: 0.9474 , 0.9300.

0.7946666666666666 0.6622807017543859
F1 score and accuracy score for training set: 0.7947 , 0.6623.
F1 score and accuracy score for test set: 0.7952 , 0.6600.

[05:15:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1.0 1.0
F1 score and accuracy score for training set: 1.0000 , 1.0000.
F1 score and accuracy score for test set: 0.9474 , 0.9300.



In [24]:
parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

clf = xgb.XGBClassifier(seed=2)

f1_scorer = make_scorer(f1_score,pos_label='H')

grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

grid_obj = grid_obj.fit(x_train,y_train)

clf = grid_obj.best_estimator_
print (clf)

f1, acc = predict_labels(clf, x_train, y_train)
print ("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
f1, acc = predict_labels(clf, x_test, y_test)
print ("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

[05:19:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:19:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:19:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:19:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

C:\Users\14372\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\14372\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\14372\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\14372\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call,

In [25]:

clf.predict(x_test)[2]

'W'